##**INSTALLING TRANSFORMERS PIPELINE**

In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.1 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


##**IMPORTING DATASETS**

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset


dataset = load_dataset("sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset sst2 downloaded and prepared to /root/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

##**DATA PREPROCESSING OR TOKENIZATION**

In [ ]:

from transformers import AutoTokenizer
bert_model = 'gchhablani/bert-base-cased-finetuned-sst2'
tokenizer = AutoTokenizer.from_pretrained(bert_model)

In [ ]:
def Tokenizatoin(sentences):
  return tokenizer( sentences['sentence'], truncation=True)

tokenizaed_dataset = dataset.map(Tokenizatoin)


  0%|          | 0/67349 [00:00<?, ?ex/s]

  0%|          | 0/872 [00:00<?, ?ex/s]

  0%|          | 0/1821 [00:00<?, ?ex/s]

In [ ]:
from transformers import DataCollatorWithPadding

collator = DataCollatorWithPadding(tokenizer=tokenizer,
                                   return_tensors='tf')

train_dataset = tokenizaed_dataset['train'].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    collate_fn=collator,
    batch_size=32,
    shuffle=True
)
validation_dataset = tokenizaed_dataset['validation'].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    collate_fn=collator,
    batch_size=32,
    shuffle=False
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[link text](https://)##**TRAIN TEST SPLIT**

In [ ]:

from transformers import AutoTokenizer
bert_model = 'gchhablani/bert-base-cased-finetuned-sst2'
tokenizer = AutoTokenizer.from_pretrained(bert_model)

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(bert_model,from_pt=True,num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


##**IMPORTING ADAM OPTIMIZER**

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers.schedules import PolynomialDecay

num_epochs = 5
num_train_steps = len(train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.,
    decay_steps=num_train_steps
)

In [ ]:
import tensorflow as tf
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(2e-5, weight_decay_rate=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer,loss=loss)

In [ ]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=5)

Epoch 1/5
2105/2105 [==============================] - 661s 304ms/step - loss: 0.0710 - val_loss: 0.2416
Epoch 2/5
2105/2105 [==============================] - 645s 307ms/step - loss: 0.0578 - val_loss: 0.2365
Epoch 3/5
2105/2105 [==============================] - 646s 307ms/step - loss: 0.0432 - val_loss: 0.3337
Epoch 4/5
2105/2105 [==============================] - 646s 307ms/step - loss: 0.0343 - val_loss: 0.3451
Epoch 5/5
2105/2105 [==============================] - 648s 308ms/step - loss: 0.0285 - val_loss: 0.3471


tf_model.h5:   0%|          | 0.00/434M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset


DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [ ]:
import numpy as np
preds = model.predict(validation_dataset)['logits']
probabilities = tf.nn.softmax(preds)
class_preds = np.argmax(probabilities, axis=1)

28/28 [==============================] - 11s 112ms/step


In [ ]:
label_names = dataset["train"].features["label"].names
label_names


['negative', 'positive']

In [ ]:
model.config.id2label = {str(i): lbl for i, lbl in enumerate(label_names)}
model.config.label2id = {lbl: str(i) for i, lbl in enumerate(label_names)}

CommitInfo(commit_url='https://huggingface.co/Ramuvannela/Stanford-Sentiment-Treebank/commit/8231463bf045e8e73a1c9e37de63db858abd0b93', commit_message='Upload config', commit_description='', oid='8231463bf045e8e73a1c9e37de63db858abd0b93', pr_url=None, pr_revision=None, pr_num=None)

##Accuracy

In [ ]:

from datasets import load_metric

metric = load_metric("glue", "mrpc")
metric.compute(predictions=class_preds, references=dataset['validation']['label'])

{'accuracy': 0.9231651376146789, 'f1': 0.9248035914702581}

In [ ]:

import numpy as np

from transformers import AutoTokenizer
bert_model = 'gchhablani/bert-base-cased-finetuned-sst2'
tokenizer = AutoTokenizer.from_pretrained(bert_model)

from transformers  import TFAutoModelForSequenceClassification
checkpoint = 'gchhablani/bert-base-cased-finetuned-sst2'
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint,from_pt=True)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


##lets Try Text-classification whether its a postive review or negative review

In [ ]:
raw_inputs = ['this is not good product']


inputs = tokenizer(raw_inputs,padding=True,return_tensors='tf')
outputs = model(**inputs)
preds = outputs.logits
probabilities = tf.nn.softmax(preds)
class_preds = np.argmax(probabilities, axis=1)
if class_preds==0:
  print("postive")
else:
  print("negative")


postive
